* Master DAC - BDLE
* Author: Mohamed-Amine Baazizi
* Affiliation: LIP6 - Faculté des Sciences - Sorbonne Université
* Email: mohamed-amine.baazizi@lip6.fr
* October 2023

# Data quality verification

https://github.com/awslabs/deequ

and its python-based version

https://github.com/awslabs/python-deequ

https://www.amazon.science/publications/automating-large-scale-data-quality-verification

https://medium.com/codex/how-to-check-data-quality-in-pyspark-8a882e45bc95


## Spark setup

In [1]:
!pip install --upgrade -q pyspark==3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 21.7 MB/s eta 0:00:00


In [2]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.0.0
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 11.0.20.1
Branch HEAD
Compiled by user ubuntu on 2020-06-06T11:32:25Z
Revision 3fdfce3120f307147244e5eaf46d61419a723d50
Url https://gitbox.apache.org/repos/asf/spark.git
Type --help for more information.


In [3]:
import os
os.environ["SPARK_VERSION"] = "3.0"

In [4]:
!pip install  --upgrade -q pydeequ

In [5]:
from pyspark.sql import SparkSession, Row
import pydeequ

spark = SparkSession.builder\
    .master("local")\
    .appName("pyDeequ")\
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

In [6]:
spark

In [7]:
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## usage

### examples for the official doc

In [8]:
df = spark.sparkContext.parallelize([
            Row(a="foo", b=1, c=5),
            Row(a="bar", b=2, c=6),
            Row(a="baz", b=3, c=None)]).toDF()
df.show()

+---+---+----+
|  a|  b|   c|
+---+---+----+
|foo|  1|   5|
|bar|  2|   6|
|baz|  3|null|
+---+---+----+



#### analyzers

In [12]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("b")) \
                    .run()
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+--------+------------+-----+
| entity|instance|        name|value|
+-------+--------+------------+-----+
|Dataset|       *|        Size|  3.0|
| Column|       b|Completeness|  1.0|
+-------+--------+------------+-----+



=> On a analysé deux métrique : la taille et la complétude de la colonne b.


=> La colonne b est complète (elle ne contient aucune valeur nulle) et le dataframe a 3 lignes.

#### profile

In [15]:
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

StandardProfiles for column: a: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 3,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 3
    },
    "histogram": [
        [
            "baz",
            1,
            0.3333333333333333
        ],
        [
            "foo",
            1,
            0.3333333333333333
        ],
        [
            "bar",
            1,
            0.3333333333333333
        ]
    ]
}
NumericProfiles for column: b: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 3,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": [
        [
            "1",
            1,
            0.3333333333333333
        ],
        [
            "2",
            1,
            0.3333333333333333
        ],
        [
            "3",
            1,
           

#### constraint suggestion

In [38]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT())\
             .run()

# Constraint Suggestions in JSON format
print(suggestionResult)

{'constraint_suggestions': [{'constraint_name': 'CompletenessConstraint(Completeness(b,None))', 'column_name': 'b', 'current_value': 'Completeness: 1.0', 'description': "'b' is not null", 'suggesting_rule': 'CompleteIfCompleteRule()', 'rule_description': 'If a column is complete in the sample, we suggest a NOT NULL constraint', 'code_for_constraint': '.isComplete("b")'}, {'constraint_name': "ComplianceConstraint(Compliance('b' has no negative values,b >= 0,None))", 'column_name': 'b', 'current_value': 'Minimum: 1.0', 'description': "'b' has no negative values", 'suggesting_rule': 'NonNegativeNumbersRule()', 'rule_description': 'If we see only non-negative numbers in a column, we suggest a corresponding constraint', 'code_for_constraint': '.isNonNegative("b")'}, {'constraint_name': 'UniquenessConstraint(Uniqueness(List(b),None))', 'column_name': 'b', 'current_value': 'ApproxDistinctness: 1.0', 'description': "'b' is unique", 'suggesting_rule': 'UniqueIfApproximatelyUniqueRule()', 'rule_

#### constraint verification

In [42]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

def toto(x):
  return x>=3

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 3) \
        .hasSize(toto)\
        .hasMin("b", lambda x: x == 0) \
        .isComplete("c")  \
        .isUnique("a")  \
        .isContainedIn("a", ["foo", "bar", "baz"]) \
        .isNonNegative("b")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+------------+-----------+------------+-----------------------------------------------------------------------------------------------------------+-----------------+-------------------------------------------------------------------+
|check       |check_level|check_status|constraint                                                                                                 |constraint_status|constraint_message                                                 |
+------------+-----------+------------+-----------------------------------------------------------------------------------------------------------+-----------------+-------------------------------------------------------------------+
|Review Check|Warning    |Warning     |SizeConstraint(Size(None))                                                                                 |Success          |                                                                   |
|Review Check|Warning    |Warning     |SizeConstraint(Size(None)

### other examples

In [43]:
!wget --no-verbose https://nuage.lip6.fr/s/89BG8HD9r3iE693/download/MLData.tgz -O /tmp/MLData.tgz

2023-11-10 16:06:06 URL:https://nuage.lip6.fr/s/89BG8HD9r3iE693/download/MLData.tgz [19397838/19397838] -> "/tmp/MLData.tgz" [1]


In [44]:
!tar -xzvf /tmp/MLData.tgz  --directory /tmp/

MLData/
MLData/._loan.csv
MLData/loan.csv
MLData/autos.csv


In [45]:
!rm  /tmp/MLData.tgz
!rm /tmp/MLData/\._loan.csv
!ls -hal /tmp/MLData

total 73M
drwxr-xr-x 2  501 staff 4.0K Nov 10 16:06 .
drwxrwxrwt 1 root root  4.0K Nov 10 16:06 ..
-rw-r--r-- 1  501 staff  66M Jan  6  2022 autos.csv
-rw-r--r-- 1  501 staff 6.8M Jan  6  2022 loan.csv


In [46]:
#load
dir = "/tmp/MLData/"
data =  spark.read\
            .format("csv").option("header", "true")\
            .option("inferSchema", "true")\
            .load(dir +"autos.csv")

In [47]:
data.count()

371824

In [48]:
sample =data.sample(0.1)
sample.count()

37494

In [49]:
sample.show()

+-------------------+--------------------+------+---------+-----+-------+-----------+------------------+---------+-------+--------+---------+-------------------+--------+-------------+-----------------+-------------------+------------+----------+-------------------+
|        dateCrawled|                name|seller|offerType|price| abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|   model|kilometer|monthOfRegistration|fuelType|        brand|notRepairedDamage|        dateCreated|nrOfPictures|postalCode|           lastSeen|
+-------------------+--------------------+------+---------+-----+-------+-----------+------------------+---------+-------+--------+---------+-------------------+--------+-------------+-----------------+-------------------+------------+----------+-------------------+
|2016-03-14 12:52:21|Jeep_Grand_Cherok...|privat|  Angebot| 9800|   test|        suv|              2004|automatik|    163|   grand|   125000|                  8|  diesel|         jeep|             nu

In [50]:
sample.printSchema()

root
 |-- dateCrawled: string (nullable = true)
 |-- name: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: integer (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- dateCreated: string (nullable = true)
 |-- nrOfPictures: integer (nullable = true)
 |-- postalCode: integer (nullable = true)
 |-- lastSeen: string (nullable = true)



#### Analyzers

In [51]:
# set these variables
_data = sample
_colName = "vehicleType"

#reuse from tutorial
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(_data) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness(_colName)) \
                    .addAnalyzer(CountDistinct(_colName)) \
                    .addAnalyzer(Distinctness(_colName))\
                    .run()


In [52]:
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show(truncate=False)

+-------+-----------+-------------+---------------------+
|entity |instance   |name         |value                |
+-------+-----------+-------------+---------------------+
|Dataset|*          |Size         |37494.0              |
|Column |vehicleType|Completeness |0.8984104123326399   |
|Column |vehicleType|CountDistinct|8.0                  |
|Column |vehicleType|Distinctness |2.3749443372420958E-4|
+-------+-----------+-------------+---------------------+



Now consider more metrics (consistency)

In [53]:
_targetColName = 'price'
_otherColName = 'powerPS'
analysisResult = AnalysisRunner(spark) \
                    .onData(_data) \
                    .addAnalyzer(Entropy(_colName))\
                    .addAnalyzer(Correlation(_otherColName,_targetColName))\
                    .run()


In [54]:
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show(truncate=False)

+-----------+-------------+-----------+--------------------+
|entity     |instance     |name       |value               |
+-----------+-------------+-----------+--------------------+
|Mutlicolumn|powerPS,price|Correlation|3.988948607907948E-4|
|Column     |vehicleType  |Entropy    |1.7703514405647869  |
+-----------+-------------+-----------+--------------------+



#### profile

In [55]:
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(_data) \
    .run()


In [ ]:
# for col, profile in result.profiles.items():
#     print(profile)

In [56]:
result.columnProfileClasses

{'StandardColumnProfile': pydeequ.profiles.StandardColumnProfile,
 'NumericColumnProfile': pydeequ.profiles.NumericColumnProfile}

In [57]:
result.profiles

{'lastSeen': <pydeequ.profiles.StandardColumnProfile at 0x7a77deadf580>,
 'name': <pydeequ.profiles.StandardColumnProfile at 0x7a77deaddfc0>,
 'yearOfRegistration': <pydeequ.profiles.NumericColumnProfile at 0x7a77deadc940>,
 'model': <pydeequ.profiles.StandardColumnProfile at 0x7a77deade8c0>,
 'abtest': <pydeequ.profiles.StandardColumnProfile at 0x7a77deadfb50>,
 'powerPS': <pydeequ.profiles.NumericColumnProfile at 0x7a77dec12680>,
 'fuelType': <pydeequ.profiles.StandardColumnProfile at 0x7a77deadf790>,
 'notRepairedDamage': <pydeequ.profiles.StandardColumnProfile at 0x7a77deaddb10>,
 'price': <pydeequ.profiles.NumericColumnProfile at 0x7a77dec13760>,
 'vehicleType': <pydeequ.profiles.StandardColumnProfile at 0x7a77deadd720>,
 'dateCrawled': <pydeequ.profiles.StandardColumnProfile at 0x7a77deadcf10>,
 'offerType': <pydeequ.profiles.StandardColumnProfile at 0x7a77dec11840>,
 'gearbox': <pydeequ.profiles.StandardColumnProfile at 0x7a77dec13880>,
 'monthOfRegistration': <pydeequ.profiles.

In [58]:

ex_result = ColumnProfilerRunner(spark) \
    .onData(_data.select('name','powerPs')) \
    .run()

In [59]:
for col, profile in ex_result.profiles.items():
    print(profile)

StandardProfiles for column: name: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 30158,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 11,
        "Unknown": 0,
        "String": 37483
    },
    "histogram": null
}
NumericProfiles for column: powerPs: {
    "completeness": 0.9999733290659839,
    "approximateNumDistinctValues": 408,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null,
    "kll": "None",
    "mean": 115.87408316219027,
    "maximum": 20000.0,
    "minimum": 0.0,
    "sum": 4344467.0,
    "stdDev": 212.6371052170788,
    "approxPercentiles": []
}


##### [TODO] import the JSON object to a DF to facilitate its processing

In [60]:
type(_data)

pyspark.sql.dataframe.DataFrame

#### constraint suggestion

In [61]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(_data) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
#print(suggestionResult)


In [62]:
suggestionResult

{'constraint_suggestions': [{'constraint_name': 'CompletenessConstraint(Completeness(lastSeen,None))',
   'column_name': 'lastSeen',
   'current_value': 'Completeness: 0.9999733290659839',
   'description': "'lastSeen' has less than 1% missing values",
   'suggesting_rule': 'RetainCompletenessRule()',
   'rule_description': 'If a column is incomplete in the sample, we model its completeness as a binomial variable, estimate a confidence interval and use this to define a lower bound for the completeness',
   'code_for_constraint': '.hasCompleteness("lastSeen", lambda x: x >= 0.99, "It should be above 0.99!")'},
  {'constraint_name': 'CompletenessConstraint(Completeness(name,None))',
   'column_name': 'name',
   'current_value': 'Completeness: 1.0',
   'description': "'name' is not null",
   'suggesting_rule': 'CompleteIfCompleteRule()',
   'rule_description': 'If a column is complete in the sample, we suggest a NOT NULL constraint',
   'code_for_constraint': '.isComplete("name")'},
  {'c

In [63]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(_data.select('fuelType','powerPs')) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
#print(suggestionResult)

In [64]:
suggestionResult


{'constraint_suggestions': [{'constraint_name': "ComplianceConstraint(Compliance('fuelType' has value range 'benzin', 'diesel', 'lpg', 'cng', 'hybrid', 'andere', 'elektro',`fuelType` IN ('benzin', 'diesel', 'lpg', 'cng', 'hybrid', 'andere', 'elektro'),None))",
   'column_name': 'fuelType',
   'current_value': 'Compliance: 1',
   'description': "'fuelType' has value range 'benzin', 'diesel', 'lpg', 'cng', 'hybrid', 'andere', 'elektro'",
   'suggesting_rule': 'CategoricalRangeRule()',
   'rule_description': 'If we see a categorical range for a column, we suggest an IS IN (...) constraint',
   'code_for_constraint': '.isContainedIn("fuelType", ["benzin", "diesel", "lpg", "cng", "hybrid", "andere", "elektro"])'},
  {'constraint_name': "ComplianceConstraint(Compliance('fuelType' has value range 'benzin', 'diesel' for at least 98.0% of values,`fuelType` IN ('benzin', 'diesel'),None))",
   'column_name': 'fuelType',
   'current_value': 'Compliance: 0.9820237904731424',
   'description': "'fue

##### [TODO] import the JSON object to a DF to facilitate its processing

#### constraint verification

In [65]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(_data) \
    .addCheck(
        check
        .isComplete('price')  \
        .isNonNegative('price')  \
        .isUnique('seller'))\
    .run()



In [68]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()#truncate=False)

+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|CompletenessConst...|          Failure|Value: 0.99997332...|
|Review Check|    Warning|     Warning|ComplianceConstra...|          Success|                    |
|Review Check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 2.66716453...|
+------------+-----------+------------+--------------------+-----------------+--------------------+

